<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cargamos-los-datos" data-toc-modified-id="Cargamos-los-datos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cargamos los datos</a></span></li><li><span><a href="#Creación-del-esquema-para-SQL" data-toc-modified-id="Creación-del-esquema-para-SQL-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creación del esquema para SQL</a></span><ul class="toc-item"><li><span><a href="#Conexión-con-el-servidor-de-SQL-y-Creación-BBDD" data-toc-modified-id="Conexión-con-el-servidor-de-SQL-y-Creación-BBDD-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Conexión con el servidor de SQL y Creación BBDD</a></span></li><li><span><a href="#Creación-de-tablas" data-toc-modified-id="Creación-de-tablas-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Creación de tablas</a></span></li><li><span><a href="#Insercción-de-datos" data-toc-modified-id="Insercción-de-datos-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Insercción de datos</a></span><ul class="toc-item"><li><span><a href="#Insercción-de-datos-en-la-tabla-de-tiempo:" data-toc-modified-id="Insercción-de-datos-en-la-tabla-de-tiempo:-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Insercción de datos en la tabla de tiempo:</a></span></li></ul></li><li><span><a href="#Insercción-de-datos-en-la-tabla-de-localidades:" data-toc-modified-id="Insercción-de-datos-en-la-tabla-de-localidades:-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Insercción de datos en la tabla de localidades:</a></span></li><li><span><a href="#Insercción-de-datos-en-la-tabla-de-mediciones:" data-toc-modified-id="Insercción-de-datos-en-la-tabla-de-mediciones:-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Insercción de datos en la tabla de mediciones:</a></span></li></ul></li></ul></div>

El siguiente paso en el proceso de ETL es cargar nuestros datos en una BBDD, en nuestro lo haremos en SQL a través de Python. 

In [1]:
import pandas as pd
import mysql.connector

# Cargamos los datos 

In [2]:
df = pd.read_pickle('files/datos_actualizados.pkl')
df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
1,6,2,15,none,0,6,55,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2


# Creación del esquema para SQL 

Lo primero que tenemos que pensar es como queremos "montar" nuestra BBDD. Nosotras tenemos toda nuestra información en un único fichero, pero lo vamos a separar en distintas tablas. Crearemos tres tablas:

- La tabla que tiene toda la información climática 


- La tabla que tiene la información sobre la localidad(ciudad y coordenadas)


- La tabla que tiene la información temporal (timepoint y fecha) 


De la misma forma que en la sesión anterior iremos creando funciones. 

## Conexión con el servidor de SQL y Creación BBDD

Lo primero que vamos a hacer es conectarnos con nuestro servidor y crear la BBDD si no existe. La función recibirá un parámetro que será el nombre de la BBDD. 

In [5]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab" # aquí tendréis que poner vuestra contraseña de MySQL
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)


In [6]:
mydb = crear_bbdd("clima")

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS clima;


Genial! Ya tenemos la conexion creada y nuestra BBDD creada. 

## Creación de tablas

Es el momento de crear las tablas y definir que columnas tendrán cada una de ellas y de que tipo serán.

Lo primero que haremos será crearnos una función que nos valga para crear o insertar datos en cualquier tabla, ya sea para esta BBDD o para cualquier otra que queramos. Esta función recibirá tres parámetros: 

- Nombre de la BBDD


- Contraseña de nuestro servidor de SQL


- La query de insertar

In [7]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)


Es el momento de definir las distintas *queries* que crearán nuestras tablas. 

1️⃣ Tabla tiempo (incluiremos las columnas de `timepoint`y `fecha` de nuestro *dataframe* )

In [8]:
tabla_tiempo = '''
CREATE TABLE IF NOT EXISTS `clima`.`tiempo` (
  `idtiempo` INT NOT NULL AUTO_INCREMENT,
  `timepoint`INT NOT NULL,
  `fecha` DATE NOT NULL,
  PRIMARY KEY (`idtiempo`))
ENGINE = InnoDB;
'''

2️⃣ Tabla_localidades (incluiremos las columnas de `latitud`, `longitud` y `ciudad`)

In [9]:
tabla_localidades = '''
CREATE TABLE IF NOT EXISTS `clima`.`localidades` (
  `idlocalidad` INT NOT NULL AUTO_INCREMENT,
  `latitud` DECIMAL(6,4) NOT NULL,
  `longitud` DECIMAL(6,4) NOT NULL,
  `ciudad` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`idlocalidad`))
ENGINE = InnoDB;
'''

3️⃣ Tabla mediciones ( que incluirá las columnas `cloudcover`, `lifted_index`, `prec_type`, `prec_amount`, `temp2m`, `rh2m`, `weather`, `wind10m.direction`, `wind10m.speed`, `seeing` y `transparency`) 

In [10]:
tabla_mediciones = '''
CREATE TABLE IF NOT EXISTS `clima`.`mediciones` (
  `idmedicion` INT NOT NULL AUTO_INCREMENT,
  `cloudcover` INT NOT NULL,
  `lifted_index` INT NOT NULL,
  `prec_type` VARCHAR(10) NOT NULL,
  `prec_amount` INT NOT NULL,
  `temp2m` INT NOT NULL,
  `rh2m` INT NOT NULL,
  `weather` VARCHAR(15) NOT NULL,
  `wind_direction` VARCHAR(15) NOT NULL,
  `wind_speed` INT NOT NULL,
  `seeing` DECIMAL NOT NULL,
  `transparency` INT NOT NULL,
  `idtiempo` INT NOT NULL,
  `idlocalidad` INT NOT NULL,
  PRIMARY KEY ( `idmedicion`, `idtiempo`, `idlocalidad`),
  INDEX `fk_mediciones_tiempo1_idx` (`idtiempo` ASC) ,
  INDEX `fk_mediciones_localidades1_idx` (`idlocalidad` ASC) ,
  CONSTRAINT `fk_mediciones_tiempo1`
    FOREIGN KEY (`idtiempo`)
    REFERENCES `clima`.`tiempo` (`idtiempo`),
  CONSTRAINT `fk_mediciones_localidades1`
    FOREIGN KEY (`idlocalidad`)
    REFERENCES `clima`.`localidades` (`idlocalidad`))
ENGINE = InnoDB;

'''

Llamamos a la función para crear cada una de las tablas de nuestra BBDD. 

In [12]:
crear_insertar_tabla("clima","AlumnaAdalab", tabla_tiempo)

In [13]:
crear_insertar_tabla("clima", "AlumnaAdalab", tabla_localidades)

In [14]:
crear_insertar_tabla("clima", "AlumnaAdalab", tabla_mediciones)

## Insercción de datos 

Hasta ahora nos hemos creado cada una de las tablas que componen nuestra BBDD, pero de momento están vacías. Lo que haremos será nutrir nuestros datos desde el dataframe. 

Para ello necesitaremos iterar por el *dataframe* usando el método `iterrows()`, su documnetación [aquí](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html).  Pero antes de ponernos manos a la obra entendamos que hace el método `iterrows` 👇🏽:

In [15]:
df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
1,6,2,15,none,0,6,55,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2


In [16]:
for indice, fila in df.iterrows():
    print(indice, "------>", fila)
    break

0 ------> timepoint                              3
cloudcover                             1
lifted_index                          15
prec_type                           none
prec_amount                            0
temp2m                                 5
rh2m                                  64
weather                       clearnight
wind10m.direction                     NW
wind10m.speed                          3
fecha                2023-01-10 00:00:00
latitud                          40.4165
longitud                        -3.70256
ciudad                            Madrid
seeing                                 5
transparency                           2
Name: 0, dtype: object


Lo que va a hacer es iterar por cada una de las filas del *dataframe*.

- El índice corresponde con el número de fila del *datafarame*


- La fila corresponde con cada fila de tal forma que nos permitirá acceder a cada elemento de cada fila por columna. Veámoslo con un ejemplo: 

In [14]:
for indice, fila in df.iterrows():
    print(fila["fecha"])
    break


2022-09-15 00:00:00


Si nos fijamos estamos extrayendo los valores de la columna `fecha` para cada fila del *dataframe* 

Una vez entendido como "funciona" el iterrows pongamonos manos a la obra. Recordamos que al inicio del jupyter nos creamos una función que nos servía tanto para crear como para insertar datos ya que bebe de una query que le pasamos. 

### Insercción de datos en la tabla de tiempo: 

En este caso solo nos interesan dos columnas de nuestro *dataframe* `timepoint` y `fecha`, por lo tanto, cuando iteremos por el dataframe solo accederemos a estas columnas. 

In [15]:
df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,3,6,none,0,16,58,pcloudynight,NW,3,2022-09-15,40.4165,-3.70256,Madrid,4,2
1,6,2,6,none,0,16,56,clearday,NW,3,2022-09-15,40.4165,-3.70256,Madrid,4,2


In [17]:
# manos a la obra 

for indice, fila in df.iterrows(): # itreamos por el dataframe.
    
    # definimos nuestra query, igual que si lo hicieramos en workbench. ⚠️ Como estamos definiendo nuestra query en varias líneas usamos las triples comillas
    # lo valores que introduciremos serán los del dataframe que estamos iterando, por lo que usaremos los formats de los strings. 
    
    query_tiempo = f"""
            INSERT INTO tiempo (timepoint, fecha) # como la primary key la hemos puesto como autoincremental no hace falta introducirla en el insert
            VALUES ( "{fila["timepoint"]}", "{fila['fecha']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
    crear_insertar_tabla("clima", "AlumnaAdalab", query_tiempo)



## Insercción de datos en la tabla de localidades:

In [17]:
df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,3,6,none,0,16,58,pcloudynight,NW,3,2022-09-15,40.4165,-3.70256,Madrid,4,2
1,6,2,6,none,0,16,56,clearday,NW,3,2022-09-15,40.4165,-3.70256,Madrid,4,2


Lo primero que tenemos que hacer es sacar los elementos únicos de nuestro *dataframe*, es decir, en esta tabla tendremos una fila por cada ciudad de la que tengamos datos. En nuestro caso, solo tenemos una ciudad por lo tanto esta tabla solo tendrá un elemento. 

Por otro lado, antes de introducir cualquier dato, tendremos que chequear si esa ciudad ya está en nuestra BBDD. 

Para chequear si esa ciudad ya existe en nuestro *dataframe* tendremos que hacer una query que nos devuelva las ciudades que tenemos. 

In [19]:
# con esta query extraermos todos los valores únicos que tenemos en nuestra BBDD para la ciudad.

def check_ciudades(contraseña, nombre_bbdd):
    
    cnx = mysql.connector.connect(user='root', password=f'{contraseña}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}")
    mycursor = cnx.cursor()


    query_existe_ciudad = f"""
            SELECT DISTINCT ciudad FROM localidades
            """
    mycursor.execute(query_existe_ciudad)
    ciudades = mycursor.fetchall()
    return ciudades

In [20]:
# qué nos devuelve esta función 

check_ciudades("AlumnaAdalab", "clima")

[]

In [21]:
# Tenemos una lista con una tupla, por lo que nos tenemos que quedarnos con el elemento 0. 

check_ciudades("AlumnaAdalab", "clima")[0]

IndexError: list index out of range

Sobre este resultado ya podremos chequear si la ciudad que queremos insertar esta en nuestra BBDD o no. Vamos manos a la obra. 

In [22]:
for indice, fila in df.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_localidad = f"""
                INSERT INTO localidades (latitud, longitud, ciudad) 
                VALUES ( "{fila["latitud"]}", "{fila['longitud']}", "{fila['ciudad']}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    ciudades = check_ciudades("AlumnaAdalab", "clima")
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
    # 
    if len(ciudades) == 0 or fila['ciudad'] not in ciudades[0]: 
        crear_insertar_tabla("clima","AlumnaAdalab", query_localidad)

    else:
        print(f"{fila['ciudad']} ya esta en nuestra BBDD")    

Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid y

## Insercción de datos en la tabla de mediciones:

Es el momento de meter en la BBDD la información de las mediciones. Y aquí la cosa se complica un poco porque tenemos que tener en cuenta que debemos vincular estos datos con las *primary keys* de las otras tablas. 

Tendremos que chequear a que ciudad corresponde cada fila de nuestra ciudad y vincularlo con el id que tiene en la BBDD de datos y lo mismo para la información de las fechas. 

In [23]:
df.head(3)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
1,6,2,15,none,0,6,55,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
2,9,1,15,none,0,8,45,clearday,NW,2,2023-01-10,40.4165,-3.70256,Madrid,3,2


In [24]:
df["seeing"] = round(df["seeing"], 2)

Lo primero que tendremos que hacer será crearnos una query que nos devuelva el `id` de la ciudad para cada fila de nuestro *dataframe* 

In [25]:
def sacar_id_ciudad(contraseña, nombre_bbdd, ciudad):
    
    # creamos la conexion con el servidor y la BBDD
    cnx = mysql.connector.connect(user='root', password= f'{contraseña}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}")
    mycursor = cnx.cursor()

    # creamos la query para sacar el id de la ciudad que queramos
    query_sacar_id = f"SELECT idlocalidad FROM localidades WHERE ciudad = '{ciudad}'"
    
    # puede que el id de la ciudad que estemos intentando insertar no este en nuestra BBDD. Por lo que vamos a hacer un try except para que no nos falle el código
    try: 
        mycursor.execute(query_sacar_id)
        id_ = mycursor.fetchall()[0][0]
        return id_
    
    except: 
        return "Sorry, no tenemos esa ciudad en la BBDD y por lo tanto no te podemos dar su id. "

In [26]:
sacar_id_ciudad("AlumnaAdalab", "clima", 'Madrid')

1

Lo mismo tendremos que hacer para las fechas: 

In [27]:
def sacar_id_fecha(contraseña, nombre_bbdd, timepoint, fecha):
        
        cnx = mysql.connector.connect(user='root', password=f'{contraseña}',
                                      host='127.0.0.1', database=f"{nombre_bbdd}")
        mycursor = cnx.cursor()


        query_sacar_id = f"SELECT idtiempo FROM tiempo WHERE timepoint = {timepoint} and fecha = '{fecha}'"
        
        try: 
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "
            

In [28]:
sacar_id_fecha("AlumnaAdalab", "clima", 3, '2022-03-16' )

'Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. '

Una vez que somos capaces de sacar los `id` para la fecha y las localidades ya podremos insertar los datos en la BBDD. 

In [29]:
for indice, fila in df.iterrows():
    
    id_ciudad = sacar_id_ciudad("AlumnaAdalab", "clima", fila['ciudad'])
    id_fecha = sacar_id_fecha("AlumnaAdalab", "clima", fila["timepoint"], fila["fecha"])
    
    query_medidas = f"""
                INSERT INTO mediciones (cloudcover, lifted_index, prec_type, prec_amount, temp2m, rh2m, weather, wind_direction, wind_speed, seeing, transparency, idtiempo, idlocalidad) 
                VALUES ({fila["cloudcover"]}, {fila['lifted_index']}, "{fila['prec_type']}", {fila['prec_amount']},{fila['temp2m']},{fila['rh2m']}, "{fila['weather']}","{fila['wind10m.direction']}",{fila['wind10m.speed']}, {fila['seeing']}, {fila['transparency']}, {id_fecha}, {id_ciudad});
                """
    crear_insertar_tabla("clima","AlumnaAdalab", query_medidas)
    

Perfecto! Ya tenemos toda la información metida en nuestra BBDD. Hemos completado el proceso de carga de nuestros datos 🥳. Ahora solo nos queda crearnos una clase para tener todo este proceso unificado. Este proceso lo tenéis en el siguiente jupyter `ETL-3.2`

**EJERCICIOS** 

En el ejercicio de ayer hicimos la llamada a la API a un nuevo producto, lo limpiamos y lo unimos todo. En la lección de hoy aprendimos a meterlo en SQL. Con los resultados de los  disintos productos, intentad crear una BBDD con distintas tablas donde cada tabla corresponda a un producto. 